In [2]:

import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms

In [3]:
class FairFaceCustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the CSV file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.labels_df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

        # Map races to integer labels
        self.race_map = {
            'White': 0,
            'Black': 1,
            'East Asian': 2,
            'Indian': 3,
            'Latino_Hispanic': 4,
            'Middle Eastern': 5,
            'Southeast Asian': 6
        }

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.labels_df.iloc[idx, 0])  # Get image path
        image = Image.open(img_name)

        race_label = self.labels_df.iloc[idx, 3]  # Get race label (2nd column in your CSV)
        label = self.race_map[race_label]  # Convert to numerical label

        if self.transform:
            image = self.transform(image)

        return image, label


In [4]:
from torch.utils.data import DataLoader
from torchvision import transforms

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet values
])

# Load the training dataset
train_dataset = FairFaceCustomDataset(csv_file='/Users/maryamsadeghi/MyMac/darmstadt/courses/DL/ethnicity/data/fairface_label_train.csv', 
                                      root_dir='/Users/maryamsadeghi/MyMac/darmstadt/courses/DL/ethnicity/data',  # Base directory for 'train' folder
                                      transform=transform)

# Load the validation dataset
val_dataset = FairFaceCustomDataset(csv_file='/Users/maryamsadeghi/MyMac/darmstadt/courses/DL/ethnicity/data/fairface_label_val.csv', 
                                    root_dir='/Users/maryamsadeghi/MyMac/darmstadt/courses/DL/ethnicity/data',  # Base directory for 'val' folder
                                    transform=transform)

# Create DataLoaders for batching
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [10]:
import torch
import torch.nn as nn
from torchvision import models

# Load a pre-trained ResNet18 model
model = models.resnet18(pretrained=True)

# Modify the final fully connected layer to classify 7 ethnicities
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 7)  # 7 classes for the 7 ethnicities

# Check if MPS (Metal Performance Shaders) is available
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS backend (Apple Silicon GPU)")
else:
    # Fallback to CPU if MPS is not available
    device = torch.device("cpu")
    print("Using CPU")
    
# Move the model to the appropriate device
model = model.to(device)


Using MPS backend (Apple Silicon GPU)


In [11]:
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [12]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()  # Set the model to training mode
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            # Track loss and accuracy
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Print epoch statistics
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')

# Train the model
train_model(model, train_loader, criterion, optimizer, num_epochs=10)


KeyboardInterrupt: 

: 

In [8]:
! pip install torch torchvision torchaudio


In [9]:
import torch
print(torch.backends.mps.is_available())


True
